![Vidi_Waterflux_Banner](https://raw.githubusercontent.com/ITC-Water-Resources/Vidi-waterflux-merch/refs/heads/main/jupyter/Vidi_Waterflux_Banner.png)
*Roelof Rietbroek, Sedigheh Karimi, Amin Shakya EGU 2025*

# Extract GLOFAS discharge points for the considered river basins

In [1]:
%load_ext autoreload
%autoreload 2


In [4]:
import os
import geopandas as gpd
import pandas as pd

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from geoslurp import GeoslurpManager
from geoslurp.tools.xarray import *
import geoslurp.tools.pandas

#local python tools
from common.config import read_config

In [5]:
conf=read_config()
datadir=conf['dataroot']

schema=conf['geoslurpschema']
basins_t="hydroshedbasins"

In [6]:
#load static gravity field from database
gsman=GeoslurpManager(dbalias='tunnelmarge',readonly_user=False)

if not gsman.conn.schemaexists(schema):
    gsman.conn.CreateSchema(schema)

## create a table which contains the GLOFAS 4.0 pixels with the highest upstream areas linked to the basins above

(takes a while but can be reused)

In [9]:
basinsgpkg=os.path.join(datadir,'hydroshed_csl03_gl_l021.gpkg')

gdfbasins=gpd.read_file(basinsgpkg)

gdfbasins.head()

#store the pandas dataframe into the geoslurp database so we can apply queries on it
gdfbasins.to_postgis(basins_t, gsman.conn.dbeng, schema=schema, if_exists='replace')

gsman.execute(f"CREATE INDEX ON {schema}.{basins_t} USING GIST(geometry)")


In [19]:
disloctable=f"{schema}.glofasv4dischlocations"

glofuparea="glofasv4.glofasupareav4"
# determine which glofas pixels are the most representative for the basins discharge locations. Take the pixels with upstream areas larger than 90% of that of the basin, and in the vicinity of the basin

tmptable="centerpoints"
#temporary table using geogrpahy type isntead of geometry
basinsgeog="basins_geog"
#minimum is 10 percent of the smallest basin considered
min_area=0.1*16035e6



qrylat=f"""DROP TABLE IF EXISTS {disloctable};
        DROP TABLE IF EXISTS {tmptable};
        CREATE TEMP TABLE {tmptable} AS (SELECT dp.x,dp.y,dp.val,dp.geom::geography FROM {glofuparea}, LATERAL ST_PixelAsCentroids(rast) AS dp 
        WHERE dp.val > {min_area});
        CREATE INDEX ON {tmptable} USING GIST(geom);
        DROP TABLE IF EXISTS {basinsgeog};
        CREATE TEMP TABLE {basinsgeog} AS (SELECT name,"ENDO" as endo, geometry::geography as geom FROM {schema}.{basins_t});
        CREATE INDEX ON {basinsgeog} USING GIST(geom);
        CREATE TABLE {disloctable} AS SELECT bas.name,bas.endo,pixs.x,pixs.y,pixs.val as upstream_area,pixs.geom FROM {basinsgeog} AS bas
        INNER JOIN LATERAL ( SELECT cp.x,cp.y,cp.val,cp.geom FROM centerpoints AS cp 
        WHERE ST_DWithin(bas.geom,cp.geom,1e3) ORDER BY cp.val DESC LIMIT 4 ) AS pixs ON True;
        """

res=gsman.execute(qrylat)



In [35]:
# read the discharge locations and write to a geopackage file

dfdisloc=gpd.GeoDataFrame.gslrp.load(gsman.conn,f"SELECT * from  {disloctable}",geom_col='geom')
dfdisloc.head()

fout=os.path.join(datadir,conf['glofasdisloc'])
dfdisloc.to_file(fout)



In [36]:
display(dfdisloc)

,name,endo,x,y,upstream_area,geom
0,SETIT,0,4358,1429,6.373512e+10,POINT (37.875 18.575)
1,SETIT,0,4357,1429,6.367669e+10,POINT (37.825 18.575)
2,SETIT,0,4356,1429,6.364749e+10,POINT (37.775 18.575)
3,SETIT,0,4356,1430,6.347212e+10,POINT (37.775 18.525)
4,GULF OF ADEN/SOMALIA,0,4573,1672,1.184632e+11,POINT (48.625 6.425)
...,...,...,...,...,...,...
479,GOBI DESERT,2,5612,953,7.298325e+10,POINT (100.575 42.375)
480,TIBETAN PLATEAU,2,5382,1169,4.499326e+10,POINT (89.075 31.575)
481,TIBETAN PLATEAU,2,5383,1168,4.351758e+10,POINT (89.125 31.625)
482,TIBETAN PLATEAU,2,5383,1167,4.349128e+10,POINT (89.125 31.675)
